In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
import pandas as pd
import numpy as np
import random
import pickle
import sklearn.ensemble
from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error

from rdkit import Chem, DataStructs
import rdkit.Chem as rkc
import rdkit.Chem.AllChem as rkac
import rdkit.Chem.Scaffolds.MurckoScaffold as mrks
from rdkit.Chem import AllChem, MACCSkeys
from rdkit.Avalon import pyAvalonTools

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("screen_df.csv")

In [4]:
df

,DrugBank ID,Name,CAS Number,Drug Groups,InChIKey,InChI,SMILES,Formula,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,ChEMBL ID,HET ID,ChemSpider ID,BindingDB ID
0,DB00006,Bivalirudin,128270-60-0,approved; investigational,OIRCOABEOLEUMC-GEJPAHFPSA-N,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,C98H138N24O33,NaN,D03136,16129704.0,46507415.0,59173.0,CHEMBL2103749,NaN,10482069.0,50248103.0
1,DB00007,Leuprolide,53714-56-0,approved; investigational,GFIJNRVAKGFPGQ-LIJARHBVSA-N,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,C59H84N16O12,C07612,D08113,NaN,46507635.0,6427.0,CHEMBL1201199,NaN,571356.0,50369395.0
2,DB00014,Goserelin,65807-02-5,approved,BLCLNMBMMGCOAS-URPVMXJPSA-N,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,C59H84N18O14,NaN,D00573,5311128.0,46507336.0,5523.0,CHEMBL1201247,NaN,4470656.0,NaN
3,DB00027,Gramicidin D,1405-97-6,approved,NDAYQJDHGXTBJL-MWWSRJDJSA-N,InChI=1S/C96H135N19O16/c1-50(2)36-71(105-79(11...,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,C96H135N19O16,NaN,D04369,45267103.0,46507412.0,NaN,CHEMBL557217,NaN,24623445.0,NaN
4,DB00035,Desmopressin,16679-58-6,approved,NFLWUMRGJYTJIN-PNIOQBSNSA-N,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,C46H64N14O12S2,C06944,D00291,NaN,NaN,4450.0,CHEMBL1429,NaN,4470602.0,50205308.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12694,DB18711,taletrectinib,1505515-69-4,investigational,DORJQZDOULKINH-QNBGGDODSA-N,InChI=1S/C23H24FN5O.C6H10O4/c1-15(25)14-30-20-...,OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...,C29H34FN5O5,NaN,NaN,NaN,NaN,NaN,CHEMBL4650361,NaN,88297530.0,NaN
12695,DB18712,Berdazimer,NaN,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12696,DB18715,Tolebrutinib,1971920-73-6,investigational,KOEUOFPEZFUWRF-LJQANCHMSA-N,InChI=1S/C26H25N5O3/c1-2-23(32)29-16-6-7-19(17...,NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...,C26H25N5O3,NaN,NaN,NaN,NaN,NaN,CHEMBL4650323,NaN,88298633.0,NaN
12697,DB18716,Enmetazobactam,1001404-83-6,approved,HFZITXBUTWITPT-YWVKMMECSA-N,InChI=1S/C11H14N4O5S/c1-11(6-14-4-3-13(2)12-14...,[H][C@@]12CC(=O)N1[C@@H](C([O-])=O)[C@](C)(CN1...,C11H14N4O5S,NaN,NaN,NaN,NaN,NaN,CHEMBL4458276,NaN,68025157.0,NaN


In [5]:
df.columns

Index(['DrugBank ID', 'Name', 'CAS Number', 'Drug Groups', 'InChIKey', 'InChI',
       'SMILES', 'Formula', 'KEGG Compound ID', 'KEGG Drug ID',
       'PubChem Compound ID', 'PubChem Substance ID', 'ChEBI ID', 'ChEMBL ID',
       'HET ID', 'ChemSpider ID', 'BindingDB ID'],
      dtype='object')

In [6]:
selection = ['DrugBank ID', 'SMILES']
df2 = df[selection]
df2

,DrugBank ID,SMILES
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
3,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
4,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
...,...,...
12694,DB18711,OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...
12695,DB18712,NaN
12696,DB18715,NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...
12697,DB18716,[H][C@@]12CC(=O)N1[C@@H](C([O-])=O)[C@](C)(CN1...


In [7]:
df2.isna().sum()

DrugBank ID      0
SMILES         774
dtype: int64

In [8]:
final_df = df2.dropna(subset=['SMILES'])

In [9]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11925 entries, 0 to 12697
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   DrugBank ID  11925 non-null  object
 1   SMILES       11925 non-null  object
dtypes: object(2)
memory usage: 279.5+ KB


In [10]:
train_df = pd.read_csv("final_df.csv")
train_df

,molecule_chembl_id,canonical_smiles,activity_class
0,CHEMBL1401841,COc1ccc2nc3cccc(OC)c3nc2c1,1
1,CHEMBL1608853,O=C(O)c1ccc2c(c1)C(=O)/C(=C\c1ccco1)C2=O,1
2,CHEMBL1429799,O=C1NN(c2ccccc2)C(=O)/C1=C\c1ccccc1OCC(=O)N1CC...,1
3,CHEMBL246446,O=C(O)c1ccc2nc(-c3ccco3)c(-c3ccco3)nc2c1,1
4,CHEMBL1383455,CCn1nc([N+](=O)[O-])c(C(C#N)c2nc3ccccc3n2C)c(C...,1
...,...,...,...
1111,CHEMBL5189529,O=C1c2ccccc2C(=O)N1CCCCSc1nnc(-c2ccc(NS(=O)(=O...,1
1112,CHEMBL5190612,Cc1ccc(S(=O)(=O)Nc2ccc(-c3nnc(SCc4ccc(C(F)(F)F...,1
1113,CHEMBL5176952,Cc1ccc(S(=O)(=O)Nc2ccc(-c3nnc(SCc4cccc(C(F)(F)...,1
1114,CHEMBL5184942,Cc1ccc(S(=O)(=O)Nc2ccc(-c3nnc(SCc4ccccc4C(F)(F...,1


In [11]:
# filtering out active molecules from training data 
train_active =  train_df[(train_df['activity_class'] == 0)]

In [12]:
train_active 

,molecule_chembl_id,canonical_smiles,activity_class
176,CHEMBL1714002,COC1=C(Br)C(=O)c2ncccc2C1=O,0
271,CHEMBL1353221,O=c1ccc2[n+]([O-])c3ccc([O-])cc3oc-2c1.[Na+],0
318,CHEMBL1570587,O=C1C(Cl)=C(NC(=O)c2ccccc2)C(=O)C(Cl)=C1NC(=O)...,0
344,CHEMBL2006418,O=[N+]([O-])c1ccc(N/N=C/c2ccc(O)c(O)c2O)c(S(=O...,0
469,CHEMBL1699845,O=C1C(Cl)=C(Oc2ccccc2)C(=O)c2ccccc21,0
...,...,...,...
1097,CHEMBL506247,O=C(OC[C@H]1O[C@@H](OC(=O)c2cc(O)c(O)c(OC(=O)c...,0
1098,CHEMBL5194226,CCCCC(=O)Nc1nc(SC(=O)CCCC)c2nc[nH]c2n1,0
1108,CHEMBL5191539,Cc1cc([N+](=O)[O-])ccc1-n1sc2ccccc2c1=O,0
1109,CHEMBL1729965,O=c1c2ccccc2sn1-c1ccccc1Cl,0


In [13]:
training_active_smiles = train_active['canonical_smiles'].to_list()

In [14]:
training_active_smiles

['COC1=C(Br)C(=O)c2ncccc2C1=O',
 'O=c1ccc2[n+]([O-])c3ccc([O-])cc3oc-2c1.[Na+]',
 'O=C1C(Cl)=C(NC(=O)c2ccccc2)C(=O)C(Cl)=C1NC(=O)c1ccccc1',
 'O=[N+]([O-])c1ccc(N/N=C/c2ccc(O)c(O)c2O)c(S(=O)(=O)N2CCOCC2)c1',
 'O=C1C(Cl)=C(Oc2ccccc2)C(=O)c2ccccc21',
 'Cl.O=C1C(Cl)=C(N2CCN(CCO)CC2)C(=O)c2c(O)ccc(O)c21',
 'COC(CNC(=O)c1ccc2c3c(onc13)-c1ccccc1C2=O)OC',
 'COC1=C(Br)C(=O)c2nc(C)ccc2C1=O',
 'O=c1ccc2c(-c3ccccc3S(=O)(=O)O)c3ccc(O)c(O)c3oc-2c1O',
 'NS(=O)(=O)c1ccc(N/N=C/c2ccc(O)c(O)c2O)c([N+](=O)[O-])c1',
 'COc1ccc(C[C@H]2C(=O)N[C@@H](CCCCN)C(=O)N(C)[C@@H](Cc3ccccc3)C(=O)N[C@@H](CCCCN)C(=O)N(C)[C@@H](CO)C(=O)N(C)[C@@H](Cc3ccc(OC)cc3)C(=O)N[C@@H](CCCCN)C(=O)N(C)[C@@H](Cc3ccc(OC)cc3)C(=O)N(C)[C@@H](Cc3ccc(OC)cc3)C(=O)N[C@@H](CCCCN)C(=O)N[C@H](C(N)=O)CSCC(=O)N[C@@H](Cc3ccc(O)cc3)C(=O)N2C)cc1',
 'O=C(Nc1nc2cc(O)c(O)cc2s1)c1ccccc1Oc1ccc2cc(O)ccc2c1',
 'O=C1C=C/C(=N/Nc2ccc([N+](=O)[O-])cc2Cl)C=C1C(=O)Nc1cccc(Cl)c1',
 'N=C(N)NCCC[C@H](NC(=O)c1ccccc1)C(=O)N[C@@H](CCCCN)C(=O)N[C@@H](C(=O)N[C@H]1CN(c2cccc

In [15]:
len(final_df.SMILES.unique())

11919

In [16]:
final_df.drop_duplicates(['SMILES'], inplace = True, ignore_index = True)

C:\Users\Ankit\AppData\Local\Temp\ipykernel_13376\2347999863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop_duplicates(['SMILES'], inplace = True, ignore_index = True)


In [17]:
final_df

,DrugBank ID,SMILES
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
3,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
4,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
...,...,...
11914,DB18708,CC1=C(COC2=CC=C(N=N2)C(=O)NC2CCOCC2)C(=NO1)C1=...
11915,DB18709,CN1C=C(C=N1)C1=CN2N=CC=C2C(=N1)C1=CN(N=C1)[C@@...
11916,DB18711,OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...
11917,DB18715,NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...


In [18]:
# # Import RDkit packages

# from rdkit import Chem
# import rdkit.Chem as rkc
# import rdkit.Chem.AllChem as rkac
# import rdkit.Chem.Scaffolds.MurckoScaffold as mrks

In [19]:
# from rdkit import Chem

# # Assuming your screening data is stored in a DataFrame called 'screening_data'
# # Assuming the SMILES column in the DataFrame is named 'SMILES'

# # Define a function to remove stereochemistry from SMILES
# def remove_stereochemistry(smiles):
#     mol = Chem.MolFromSmiles(smiles)
#     if mol is None:
#         return None
#     Chem.RemoveStereochemistry(mol)
#     return Chem.MolToSmiles(mol)

# # Apply the function to the SMILES column
# final_df['SMILES'] = final_df['SMILES'].apply(remove_stereochemistry)

# # Remove rows with invalid SMILES after removing stereochemistry
# final_df = final_df.dropna(subset=['SMILES'])

# # Optionally, reset the index of the DataFrame
# final_df.reset_index(drop=True, inplace=True)

In [20]:
final_df

,DrugBank ID,SMILES
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
3,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
4,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
...,...,...
11914,DB18708,CC1=C(COC2=CC=C(N=N2)C(=O)NC2CCOCC2)C(=NO1)C1=...
11915,DB18709,CN1C=C(C=N1)C1=CN2N=CC=C2C(=N1)C1=CN(N=C1)[C@@...
11916,DB18711,OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...
11917,DB18715,NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...


In [21]:
# for this case we also declare compounds that are too large or too small as invalid

def is_invalid(smi):

    global i
    if smi:
        # setting sanitize to false avoids explicit valence error
        # setting it to true gives the error and can be used to count invalid mols
        mol = rkc.MolFromSmiles(smi, sanitize = True)
        if mol is None:
            return int(1)
        elif (len(smi)  > 120) or (len(smi) < 20):
            return int(1)
        else:
            pass
        return int(0)

In [22]:
# create a series of valid/invalid values. 0 = valid, 1 = invalid
invalid_list = list(map(is_invalid, final_df['SMILES']))

[17:58:32] Explicit valence for atom # 13 Cl, 5, is greater than permitted
[17:58:32] SMILES Parse Error: syntax error while parsing: OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]
[17:58:32] SMILES Parse Error: Failed parsing SMILES 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]' for input: 'OS(O)(O)C1=CC=C(C=C1)C-1=C2\C=CC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC=C(C=C1)S(O)(O)O)C1=CC=C(C=C1)S([O-])([O-])[O-])\C1=CC=C(C=C1)S(O)(O)[O-]'
[17:58:32] Explicit valence for atom # 19 O, 3, is greater than permitted
[17:58:33] Explicit valence for atom # 0 O, 3, is greater than permitted
[17:58:33] Unusual charge on atom 0 number of radical electrons set to zero
[17:58:33] Explicit valence for atom # 4 F, 2, is greater than permitted
[

In [23]:
print('The number of invalid SMILES in the dataframe is: ' + str(sum(invalid_list)))

The number of invalid SMILES in the dataframe is: 1696


In [24]:
invalid_series = pd.Series(invalid_list, name = 'Invalid')

final_df = pd.concat([final_df, invalid_series], axis = 1)

In [25]:
final_df['Invalid'].value_counts()

Invalid
0    10223
1     1696
Name: count, dtype: int64

In [26]:
# create anew df where invalid == 0
clean_valid_df = final_df[final_df['Invalid'] == 0]
clean_valid_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10223 entries, 10 to 11918
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   DrugBank ID  10223 non-null  object
 1   SMILES       10223 non-null  object
 2   Invalid      10223 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 319.5+ KB


In [27]:
# reset index since 'clean_valid_df' is a subset of 'clean_df'
clean_valid_df.reset_index(drop=True, inplace=True)

#drop 'invalid' column
clean_valid_df = clean_valid_df.drop(['Invalid'], axis = 1)

clean_valid_df.tail()

,DrugBank ID,SMILES
10218,DB18708,CC1=C(COC2=CC=C(N=N2)C(=O)NC2CCOCC2)C(=NO1)C1=...
10219,DB18709,CN1C=C(C=N1)C1=CN2N=CC=C2C(=N1)C1=CN(N=C1)[C@@...
10220,DB18711,OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...
10221,DB18715,NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...
10222,DB18716,[H][C@@]12CC(=O)N1[C@@H](C([O-])=O)[C@](C)(CN1...


In [28]:
# remove duplicates
clean_valid_df.drop_duplicates(subset = ['SMILES'], inplace = True)

#reset index, check info
clean_valid_df.reset_index(drop=True, inplace=True)
clean_valid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10223 entries, 0 to 10222
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   DrugBank ID  10223 non-null  object
 1   SMILES       10223 non-null  object
dtypes: object(2)
memory usage: 159.9+ KB


In [29]:
screen_clean_df = clean_valid_df

In [30]:
screen_clean_df

,DrugBank ID,SMILES
0,DB00114,CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O
1,DB00116,NC1=NC(=O)C2=C(NCC(CNC3=CC=C(C=C3)C(=O)N[C@@H]...
2,DB00117,N[C@@H](CC1=CNC=N1)C(O)=O
3,DB00118,C[S+](CC[C@H](N)C([O-])=O)C[C@H]1O[C@H]([C@H](...
4,DB00120,N[C@@H](CC1=CC=CC=C1)C(O)=O
...,...,...
10218,DB18708,CC1=C(COC2=CC=C(N=N2)C(=O)NC2CCOCC2)C(=NO1)C1=...
10219,DB18709,CN1C=C(C=N1)C1=CN2N=CC=C2C(=N1)C1=CN(N=C1)[C@@...
10220,DB18711,OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...
10221,DB18715,NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...


In [31]:
# Step 1: Extract SMILES sequences from training data
training_smiles = set(train_df['canonical_smiles'])

# Step 2: Remove duplicates from the downloaded database
db_df_unique = screen_clean_df[~screen_clean_df['SMILES'].isin(training_smiles)].copy()
db_df_unique.reset_index(drop=True, inplace=True)


In [32]:
db_df_unique

,DrugBank ID,SMILES
0,DB00114,CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O
1,DB00116,NC1=NC(=O)C2=C(NCC(CNC3=CC=C(C=C3)C(=O)N[C@@H]...
2,DB00117,N[C@@H](CC1=CNC=N1)C(O)=O
3,DB00118,C[S+](CC[C@H](N)C([O-])=O)C[C@H]1O[C@H]([C@H](...
4,DB00120,N[C@@H](CC1=CC=CC=C1)C(O)=O
...,...,...
10218,DB18708,CC1=C(COC2=CC=C(N=N2)C(=O)NC2CCOCC2)C(=NO1)C1=...
10219,DB18709,CN1C=C(C=N1)C1=CN2N=CC=C2C(=N1)C1=CN(N=C1)[C@@...
10220,DB18711,OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...
10221,DB18715,NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...


In [33]:
screen_db = db_df_unique

In [34]:
screen_db

,DrugBank ID,SMILES
0,DB00114,CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O
1,DB00116,NC1=NC(=O)C2=C(NCC(CNC3=CC=C(C=C3)C(=O)N[C@@H]...
2,DB00117,N[C@@H](CC1=CNC=N1)C(O)=O
3,DB00118,C[S+](CC[C@H](N)C([O-])=O)C[C@H]1O[C@H]([C@H](...
4,DB00120,N[C@@H](CC1=CC=CC=C1)C(O)=O
...,...,...
10218,DB18708,CC1=C(COC2=CC=C(N=N2)C(=O)NC2CCOCC2)C(=NO1)C1=...
10219,DB18709,CN1C=C(C=N1)C1=CN2N=CC=C2C(=N1)C1=CN(N=C1)[C@@...
10220,DB18711,OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...
10221,DB18715,NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...


In [35]:
X_train = screen_db['SMILES']

In [36]:
X_train

0                          CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O
1        NC1=NC(=O)C2=C(NCC(CNC3=CC=C(C=C3)C(=O)N[C@@H]...
2                                N[C@@H](CC1=CNC=N1)C(O)=O
3        C[S+](CC[C@H](N)C([O-])=O)C[C@H]1O[C@H]([C@H](...
4                              N[C@@H](CC1=CC=CC=C1)C(O)=O
                               ...                        
10218    CC1=C(COC2=CC=C(N=N2)C(=O)NC2CCOCC2)C(=NO1)C1=...
10219    CN1C=C(C=N1)C1=CN2N=CC=C2C(=N1)C1=CN(N=C1)[C@@...
10220    OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...
10221    NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...
10222    [H][C@@]12CC(=O)N1[C@@H](C([O-])=O)[C@](C)(CN1...
Name: SMILES, Length: 10223, dtype: object

In [37]:
# convers a list of SMILEs to a list of mols
def smi_to_mols(smi_list):
    mols_list = [Chem.MolFromSmiles(smile) for smile in smi_list]
    return mols_list


# returns morgan fingerprints as a 2D array for a list of SMILE strings
def get_morgan_fingerprints(smi_list, size, radius, useFeatures, useCounts=True):
        mols = smi_to_mols(smi_list)
        fps = [AllChem.GetMorganFingerprint(mol, radius, useCounts=useCounts, useFeatures=useFeatures) for mol in mols]
        fps_arr = np.zeros((len(fps), size), np.int32)
        for i, fp in enumerate(fps):
            for index, value in fp.GetNonzeroElements().items():
                n_index = index % size
                fps_arr[i, n_index] += int(value)
        return fps_arr

In [38]:
# conver series to a list
X_train_list = X_train.tolist()

In [39]:
# get fingerprints for training and test data
X_train_arr = get_morgan_fingerprints(X_train_list, size=2048, radius=2, useFeatures=True, useCounts=True)

[17:58:49] WARNING: not removing hydrogen atom without neighbors
[17:58:49] WARNING: not removing hydrogen atom without neighbors


In [40]:
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

In [41]:
import joblib

# Load the pickled StandardScaler object from file
scaler = joblib.load('scaler_model.pkl')



In [42]:
X_screen = scaler.transform(X_train_arr)

In [43]:
X_screen

array([[-0.56698713,  0.81958233, -0.45075486, ..., -0.13600505,
        -0.04740455, -0.0951303 ],
       [-0.06140314,  1.64100643,  0.04017717, ..., -0.13600505,
        -0.04740455, -0.0951303 ],
       [-0.90404312, -0.00184176, -1.43261893, ..., -0.13600505,
        -0.04740455, -0.0951303 ],
       ...,
       [ 0.27565286,  0.81958233, -0.45075486, ..., -0.13600505,
        -0.04740455, -0.0951303 ],
       [ 0.27565286, -0.82326585, -0.45075486, ..., -0.13600505,
        -0.04740455, -0.0951303 ],
       [ 0.44418085, -0.82326585,  0.53110921, ..., -0.13600505,
        -0.04740455, -0.0951303 ]])

In [44]:
pca_model = joblib.load('pca_model.pkl')

In [45]:
screen_cmpts = pca_model.transform(X_screen)

In [46]:
screen_cmpts.shape

(10223, 502)

# **PREDICTION**

In [47]:
import joblib

In [48]:
import pandas as pd
# from sklearn.externals import joblib

def load_and_predict_models(screen_cmpts):
    # Load KNN model
    knn_model = joblib.load('knn_model.pkl')

    # Load SVM model
    svm_model = joblib.load('svc_model.pkl')

    # Load XGBoost model
    xgb_model = joblib.load('xgb_model.pkl')

    # # Load Random Forest model
    # rf_model = joblib.load('rf_model_25.pkl')

    # Make predictions using each model
    knn_preds = knn_model.predict(screen_cmpts)
    svm_preds = svm_model.predict(screen_cmpts)
    xgb_preds = xgb_model.predict(screen_cmpts)
    # rf_preds = rf_model.predict(screen_cmpts)

    # Create DataFrames for predictions
    knn_df = pd.DataFrame({'Prediction': knn_preds})
    svm_df = pd.DataFrame({'Prediction': svm_preds})
    xgb_df = pd.DataFrame({'Prediction': xgb_preds})
    # rf_df = pd.DataFrame({'Prediction': rf_preds})

    # Save predictions to CSV files
    knn_df.to_csv('knn_predictions.csv', index=False)
    svm_df.to_csv('svm_predictions.csv', index=False)
    xgb_df.to_csv('xgb_predictions.csv', index=False)
    # rf_df.to_csv('rf_predictions.csv', index=False)


In [49]:
load_and_predict_models(screen_cmpts)


In [50]:
import pandas as pd

# Load each prediction CSV file
knn_preds = pd.read_csv('knn_predictions.csv')
svm_preds = pd.read_csv('svm_predictions.csv')
xgb_preds = pd.read_csv('xgb_predictions.csv')
# rf_preds = pd.read_csv('rf_predictions.csv')

# Combine predictions into a single DataFrame with each model's predictions in separate columns
combined_preds = pd.concat([knn_preds, svm_preds, xgb_preds], axis=1)

# Rename columns with model names
combined_preds.columns = ['KNN', 'SVM', 'XGBoost']

# Save the combined predictions to a CSV file
combined_preds.to_csv('combined_predictions.csv', index=False)


In [51]:
combined_preds

,KNN,SVM,XGBoost
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
...,...,...,...
10218,1,1,1
10219,1,1,1
10220,1,1,1
10221,1,1,1


In [52]:
combined_preds = pd.read_csv('combined_predictions.csv')

In [53]:
pred_df = pd.concat([combined_preds, screen_db], axis = 1)

In [54]:
pred_df

,KNN,SVM,XGBoost,DrugBank ID,SMILES
0,1,1,1,DB00114,CC1=NC=C(COP(O)(O)=O)C(C=O)=C1O
1,1,1,1,DB00116,NC1=NC(=O)C2=C(NCC(CNC3=CC=C(C=C3)C(=O)N[C@@H]...
2,1,1,1,DB00117,N[C@@H](CC1=CNC=N1)C(O)=O
3,1,1,1,DB00118,C[S+](CC[C@H](N)C([O-])=O)C[C@H]1O[C@H]([C@H](...
4,1,1,1,DB00120,N[C@@H](CC1=CC=CC=C1)C(O)=O
...,...,...,...,...,...
10218,1,1,1,DB18708,CC1=C(COC2=CC=C(N=N2)C(=O)NC2CCOCC2)C(=NO1)C1=...
10219,1,1,1,DB18709,CN1C=C(C=N1)C1=CN2N=CC=C2C(=N1)C1=CN(N=C1)[C@@...
10220,1,1,1,DB18711,OC(=O)CCCCC(O)=O.C[C@@H](N)COC1=CC=C(C=C1)C1=C...
10221,1,1,1,DB18715,NC1=C2N(C(=O)N([C@@H]3CCCN(C3)C(=O)C=C)C2=CC=N...


In [55]:
combined_preds.shape

(10223, 3)

In [56]:
screen_db.shape

(10223, 2)

In [57]:
zero_predictions = pred_df[((pred_df['KNN'] == 0) | (pred_df['SVM'] == 0)) | (pred_df['XGBoost'] == 0)]

# Print or use the filtered DataFrame
zero_predictions

,KNN,SVM,XGBoost,DrugBank ID,SMILES
6,0,1,1,DB00125,N[C@@H](CCCNC(N)=N)C(O)=O
136,0,1,1,DB00278,C[C@@H]1CCN([C@H](C1)C(O)=O)C(=O)[C@H](CCCNC(N...
164,1,1,0,DB00310,NS(=O)(=O)C1=C(Cl)C=CC(=C1)C1(O)NC(=O)C2=CC=CC...
165,0,1,1,DB00311,CCOC1=CC2=C(C=C1)N=C(S2)S(N)(=O)=O
238,0,1,1,DB00398,CNC(=O)C1=NC=CC(OC2=CC=C(NC(=O)NC3=CC(=C(Cl)C=...
...,...,...,...,...,...
10075,0,1,1,DB18060,CC1=CC(=CC=N1)C1=NC(=CO1)C(=O)NC1=C(N=C2N=C(OC...
10083,0,1,1,DB18068,ONC(=O)C1=NOC(CCNC(=O)C2=CC=C(Cl)C(Cl)=C2)=C1
10126,1,1,0,DB18151,NC(CO)(CO)CCC1=CC=C(SC2=CC=CC(OCC3=CC=CC=C3)=C...
10172,0,1,1,DB18345,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](CO)C(=O)N[C@@H...


In [59]:
zero_predictions.to_csv("active_molecules_1.csv", index = False)

### **So, we got 212 molecules which has been classified as active by either of the model**
### **Now we'll perform virtual screening on these 212 molecules**
> **Maestro** is used for virtual screening

In [58]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

def calculate_lipinski_properties(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return None
    
    # Calculate descriptors
    mw = Descriptors.MolWt(molecule)
    logp = Descriptors.MolLogP(molecule)
    hbd = Descriptors.NumHDonors(molecule)
    hba = Descriptors.NumHAcceptors(molecule)
    
    # Check Lipinski's Rule of Five
    lipinski_rule = {
        "Molecular Weight": mw,
        "LogP": logp,
        "Hydrogen Bond Donors": hbd,
        "Hydrogen Bond Acceptors": hba
    }
    
    # Check if molecule passes Lipinski's Rule of Five
    passes_rule = all([
        mw <= 500,
        logp <= 5,
        hbd <= 5,
        hba <= 10
    ])
    
    return lipinski_rule, passes_rule

In [59]:
# Apply Lipinski's Rule of Five calculation to each SMILES string
zero_predictions["Lipinski_Properties"] = zero_predictions["SMILES"].apply(lambda x: calculate_lipinski_properties(x))

# Extract results into separate columns
zero_predictions["Molecular_Weight"] = zero_predictions["Lipinski_Properties"].apply(lambda x: x[0]["Molecular Weight"] if x else None)
zero_predictions["LogP"] = zero_predictions["Lipinski_Properties"].apply(lambda x: x[0]["LogP"] if x else None)
zero_predictions["Hydrogen_Bond_Donors"] = zero_predictions["Lipinski_Properties"].apply(lambda x: x[0]["Hydrogen Bond Donors"] if x else None)
zero_predictions["Hydrogen_Bond_Acceptors"] = zero_predictions["Lipinski_Properties"].apply(lambda x: x[0]["Hydrogen Bond Acceptors"] if x else None)
zero_predictions["Passes_Lipinski_Rule"] = zero_predictions["Lipinski_Properties"].apply(lambda x: x[1] if x else None)

# Drop the temporary "Lipinski_Properties" column
zero_predictions.drop(columns=["Lipinski_Properties"], inplace=True)


C:\Users\Ankit\AppData\Local\Temp\ipykernel_12152\4171192739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_predictions["Lipinski_Properties"] = zero_predictions["SMILES"].apply(lambda x: calculate_lipinski_properties(x))
C:\Users\Ankit\AppData\Local\Temp\ipykernel_12152\4171192739.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zero_predictions["Molecular_Weight"] = zero_predictions["Lipinski_Properties"].apply(lambda x: x[0]["Molecular Weight"] if x else None)
C:\Users\Ankit\AppData\Local\Te

In [60]:
zero_predictions

,KNN,SVM,XGBoost,DrugBank ID,SMILES,Molecular_Weight,LogP,Hydrogen_Bond_Donors,Hydrogen_Bond_Acceptors,Passes_Lipinski_Rule
657,0,1,0,DB00878,ClC1=CC=C(NC(=N)NC(=N)NCCCCCCNC(=N)NC(=N)NC2=C...,505.458000,4.17528,10,4,False
1249,1,0,0,DB01624,OCCN1CCN(CC\C=C2\C3=C(SC4=C2C=C(Cl)C=C4)C=CC=C...,400.975000,4.23630,1,4,True
5238,0,1,0,DB07844,[H][C@@](C)(N1CC[C@]([H])(NS(=O)(=O)C2=CC3=C(S...,471.988000,1.68120,1,6,True
7122,0,1,0,DB12042,CC1=CC=C2C=C(SC2=C1)C(=O)NC1(CCCC1)C(=O)N[C@H]...,644.882000,5.23462,3,6,False
7607,0,1,0,DB12692,NCCCNCCCCNC(=O)C(O)NC(=O)CCCCCCNC(N)=N,387.529000,-1.31063,8,6,False
7815,0,1,0,DB12991,NCCCNCCCCNC(=O)[C@H](O)NC(=O)CCCCCCNC(N)=N,387.529000,-1.31063,8,6,False
8471,1,0,0,DB13841,[H]C(CCN1CCN(CCO)CC1)=C1C2=C(SC3=C1C=C(Cl)C=C3...,400.975000,4.23630,1,4,True
9058,0,1,0,DB15199,N[C@@H](CCCNC(N)=N)C(=O)NCCCN1CCN(CCCNC(=O)[C@...,512.708000,-3.20226,10,8,False
10203,1,0,0,DB18523,O=[186Re+3].CCN(CC)CCN(CCS)CC[S-].CCN(CC)CCN(C...,671.829986,1.70960,1,9,False


In [61]:
active_mols = zero_predictions[(zero_predictions['Passes_Lipinski_Rule'] == True)]

In [62]:
active_mols.reset_index(inplace=True)

In [63]:
active_mols

,index,KNN,SVM,XGBoost,DrugBank ID,SMILES,Molecular_Weight,LogP,Hydrogen_Bond_Donors,Hydrogen_Bond_Acceptors,Passes_Lipinski_Rule
0,1249,1,0,0,DB01624,OCCN1CCN(CC\C=C2\C3=C(SC4=C2C=C(Cl)C=C4)C=CC=C...,400.975,4.2363,1,4,True
1,5238,0,1,0,DB07844,[H][C@@](C)(N1CC[C@]([H])(NS(=O)(=O)C2=CC3=C(S...,471.988,1.6812,1,6,True
2,8471,1,0,0,DB13841,[H]C(CCN1CCN(CCO)CC1)=C1C2=C(SC3=C1C=C(Cl)C=C3...,400.975,4.2363,1,4,True


In [65]:
active_mols.to_csv("lipinski_tru_mols.csv", index = False)

## **THE End**
# **😁😁😁😁😁😁😁😁😁😁😁😁😁😁**

In [3]:
import pandas as pd

In [4]:
combined_df = pd.read_csv("combined_predictions.csv")

In [5]:
combined_df

,KNN,SVM,XGBoost
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
...,...,...,...
10218,1,1,1
10219,1,1,1
10220,1,1,1
10221,1,1,1


In [8]:
knn_active = combined_df[combined_df['KNN'] == 0]

In [9]:
knn_active.to_csv("knn_active.csv", index = True)

In [10]:
combined_df[combined_df['SVM'] == 0]

,KNN,SVM,XGBoost
655,0,0,1
1249,1,0,0
2858,1,0,1
8471,1,0,0
9743,1,0,1
9825,1,0,1
10203,1,0,0


In [11]:
combined_df[combined_df['XGBoost'] == 0]

,KNN,SVM,XGBoost
164,1,1,0
366,1,1,0
487,1,1,0
564,1,1,0
657,0,1,0
...,...,...,...
9816,1,1,0
9955,1,1,0
9957,1,1,0
10126,1,1,0


In [133]:
training_active_smiles

['COC1=C(Br)C(=O)c2ncccc2C1=O',
 'O=c1ccc2[n+]([O-])c3ccc([O-])cc3oc-2c1.[Na+]',
 'O=C1C(Cl)=C(NC(=O)c2ccccc2)C(=O)C(Cl)=C1NC(=O)c1ccccc1',
 'O=[N+]([O-])c1ccc(N/N=C/c2ccc(O)c(O)c2O)c(S(=O)(=O)N2CCOCC2)c1',
 'O=C1C(Cl)=C(Oc2ccccc2)C(=O)c2ccccc21',
 'Cl.O=C1C(Cl)=C(N2CCN(CCO)CC2)C(=O)c2c(O)ccc(O)c21',
 'COC(CNC(=O)c1ccc2c3c(onc13)-c1ccccc1C2=O)OC',
 'COC1=C(Br)C(=O)c2nc(C)ccc2C1=O',
 'O=c1ccc2c(-c3ccccc3S(=O)(=O)O)c3ccc(O)c(O)c3oc-2c1O',
 'NS(=O)(=O)c1ccc(N/N=C/c2ccc(O)c(O)c2O)c([N+](=O)[O-])c1',
 'COc1ccc(C[C@H]2C(=O)N[C@@H](CCCCN)C(=O)N(C)[C@@H](Cc3ccccc3)C(=O)N[C@@H](CCCCN)C(=O)N(C)[C@@H](CO)C(=O)N(C)[C@@H](Cc3ccc(OC)cc3)C(=O)N[C@@H](CCCCN)C(=O)N(C)[C@@H](Cc3ccc(OC)cc3)C(=O)N(C)[C@@H](Cc3ccc(OC)cc3)C(=O)N[C@@H](CCCCN)C(=O)N[C@H](C(N)=O)CSCC(=O)N[C@@H](Cc3ccc(O)cc3)C(=O)N2C)cc1',
 'O=C(Nc1nc2cc(O)c(O)cc2s1)c1ccccc1Oc1ccc2cc(O)ccc2c1',
 'O=C1C=C/C(=N/Nc2ccc([N+](=O)[O-])cc2Cl)C=C1C(=O)Nc1cccc(Cl)c1',
 'N=C(N)NCCC[C@H](NC(=O)c1ccccc1)C(=O)N[C@@H](CCCCN)C(=O)N[C@@H](C(=O)N[C@H]1CN(c2cccc

In [134]:
predicted_active_smiles = active_mols["SMILES"].to_list()

In [135]:
predicted_active_smiles

['OCCN1CCN(CC\\C=C2\\C3=C(SC4=C2C=C(Cl)C=C4)C=CC=C3)CC1',
 '[H][C@@](C)(N1CC[C@]([H])(NS(=O)(=O)C2=CC3=C(S2)C=C(Cl)C=C3)C1=O)C(=O)N1CCOCC1',
 '[H]C(CCN1CCN(CCO)CC1)=C1C2=C(SC3=C1C=C(Cl)C=C3)C=CC=C2']

In [136]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs import TanimotoSimilarity


# Convert training active molecules to Morgan fingerprints
training_active_fps = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles), 2, nBits=1024) for smiles in training_active_smiles]

# Example DataFrame containing predicted active molecules

# Convert predicted active molecules to Morgan fingerprints
predicted_active_fps = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smiles), 2, nBits=1024) for smiles in active_mols["SMILES"]]

# Calculate Tanimoto similarity
similar_molecules = []
for pred_fp, pred_smiles in zip(predicted_active_fps, active_mols["SMILES"]):
    for train_fp, train_smiles in zip(training_active_fps, training_active_smiles):
        similarity = TanimotoSimilarity(pred_fp, train_fp)
        if similarity >= 0.3:
            similar_molecules.append((pred_smiles, train_smiles, similarity))

# Convert results to DataFrame
similar_molecules_df = pd.DataFrame(similar_molecules, columns=["Predicted_SMILES", "Training_SMILES", "Tanimoto_Similarity"])

# Print similar molecules
print("Molecules similar to training active molecules:")
similar_molecules_df


Molecules similar to training active molecules:


,Predicted_SMILES,Training_SMILES,Tanimoto_Similarity


In [137]:
similar_molecules_df

,Predicted_SMILES,Training_SMILES,Tanimoto_Similarity


In [138]:
similar_molecules_df.to_csv("tani_sim_mol.csv", index = False)

In [139]:
active_mols

,index,KNN,SVM,XGBoost,DrugBank ID,SMILES,Molecular_Weight,LogP,Hydrogen_Bond_Donors,Hydrogen_Bond_Acceptors,Passes_Lipinski_Rule
0,1249,1,0,0,DB01624,OCCN1CCN(CC\C=C2\C3=C(SC4=C2C=C(Cl)C=C4)C=CC=C...,400.975,4.2363,1,4,True
1,5238,0,1,0,DB07844,[H][C@@](C)(N1CC[C@]([H])(NS(=O)(=O)C2=CC3=C(S...,471.988,1.6812,1,6,True
2,8471,1,0,0,DB13841,[H]C(CCN1CCN(CCO)CC1)=C1C2=C(SC3=C1C=C(Cl)C=C3...,400.975,4.2363,1,4,True


In [138]:
a = active_mols[active_mols["SMILES"] == "Cc1oncc1C(=O)Nc1ccc(C(F)(F)F)cc1"]

In [139]:
b = active_mols[active_mols["SMILES"] == "CC(C)(Oc1ccc(CCNC(=O)c2ccc(Cl)cc2)cc1)C(=O)O"]

In [140]:
c = active_mols[active_mols["SMILES"] == "CCC(Cc1ccc(OC)c(CNC(=O)c2ccc(C(F)(F)F)cc2F)c1)C(=O)O"]

In [143]:
abc = pd.concat([a,b,c], axis = 0)

In [146]:
abc

,KNN,SVM,XGBoost,RandomForest,DrugBank ID,SMILES,Molecular_Weight,LogP,Hydrogen_Bond_Donors,Hydrogen_Bond_Acceptors,Passes_Lipinski_Rule
3,0,0,0,0,DB01097,Cc1oncc1C(=O)Nc1ccc(C(F)(F)F)cc1,270.210,3.25412,1,3,True
4,0,0,0,0,DB01393,CC(C)(Oc1ccc(CCNC(=O)c2ccc(Cl)cc2)cc1)C(=O)O,361.825,3.55450,2,3,True
21,0,0,0,0,DB07070,CCC(Cc1ccc(OC)c(CNC(=O)c2ccc(C(F)(F)F)cc2F)c1)...,427.394,4.43640,2,3,True


In [145]:
abc.drop(columns=['index'], inplace=True)

In [147]:
abc.to_csv("mols_2_doc.csv", index = False)